In [1]:
import requests
import time
import numpy as np
import pandas as pd
import copy
from keras.layers import Input, Embedding, Flatten, Dot, Concatenate, Dropout, Dense
from keras.models import Model

Using TensorFlow backend.


In [2]:
#attributes 
USER_ID = 'IAHIZPIW80WPGDW7P7JE'
BASE_URL = "http://35.180.178.243"
url_reset = BASE_URL + "/reset"
url_predict = BASE_URL + "/predict"
params = {"user_id" : USER_ID}

def load_data(url_reset,params) :
    r = requests.get(url=url_reset, params=params)
    data = r.json()
    return data

data = load_data(url_reset,params)
nb_users = data["nb_users"]
nb_items = data["nb_items"]
next_state = data["next_state"]

data.keys()

dict_keys(['action_history', 'nb_items', 'nb_users', 'next_state', 'rewards_history', 'state_history'])

In [3]:
blacklisted_set = set(("next_state","nb_items","nb_users"))
new_dict = {key : value for key, value in data.items() if key not in blacklisted_set}

df = pd.DataFrame(new_dict)
df.head()

,action_history,rewards_history,state_history
0,28,0.000000,"[[72, 0, 727.1492339535973, 1.5305767414852895..."
1,3,0.000000,"[[60, 0, 727.1492339535973, 0.32787752496028, ..."
2,21,0.000000,"[[10, 0, 727.1492339535973, 0.5330189102229869..."
3,1,938.734391,"[[97, 0, 727.1492339535973, 2.080141842884405,..."
4,16,0.000000,"[[42, 0, 727.1492339535973, 1.2070173651651273..."


In [4]:
df.describe()

,action_history,rewards_history
count,200.000000,200.000000
mean,14.600000,125.388172
std,8.789555,257.508716
min,0.000000,0.000000
25%,7.000000,0.000000
50%,15.000000,0.000000
75%,22.000000,0.000000
max,29.000000,938.734391


In [5]:
df[df['rewards_history']==0].count()

action_history     151
rewards_history    151
state_history      151
dtype: int64

In [6]:
users_items_df = []
for i in range(df.shape[0]):
    users_items_df += df['state_history'][i] 
    
users_items_df = pd.DataFrame(users_items_df,columns = ['user_id','item_id',
                                                        'price','v1','v2','v3','v4','v5'])
users_items_df[users_items_df['user_id']== 26]

,user_id,item_id,price,v1,v2,v3,v4,v5
1828,26,0,727.149234,-0.123659,1.574149,1.541600,1.353384,0.915129
1829,26,1,938.734391,-0.123659,1.574149,0.891951,1.329071,1.466257
1830,26,2,849.991550,-0.123659,1.574149,-1.300179,0.989033,0.424112
1831,26,3,53.788465,-0.123659,1.574149,1.821206,-0.593361,0.583917
1832,26,4,199.061066,-0.123659,1.574149,1.408624,-0.030410,3.335690
1833,26,5,631.763552,-0.123659,1.574149,0.300533,1.266337,1.006449
1834,26,6,340.159543,-0.123659,1.574149,-0.010489,0.151493,0.931144
1835,26,7,242.491997,-0.123659,1.574149,0.495546,0.672426,0.787121
1836,26,8,195.664296,-0.123659,1.574149,2.649243,2.343519,2.664486
1837,26,9,904.735857,-0.123659,1.574149,1.757570,0.537739,1.068608


In [7]:
pos_df = df[df["rewards_history"] > 0].reset_index(drop=True)
neg_df = df[df["rewards_history"] == 0].reset_index(drop=True)

In [8]:
pos_df["user_id"] = [pos_df["state_history"][i][0][0] for i in range(pos_df.shape[0])]
pos_df["item_id_action"] = [pos_df["state_history"][i][pos_df["action_history"][i]][1] for i in range(pos_df.shape[0])]
pos_df.head()## There are some users that have more than one positive item

,action_history,rewards_history,state_history,user_id,item_id_action
0,1,938.734391,"[[97, 0, 727.1492339535973, 2.080141842884405,...",97,1
1,26,445.634777,"[[43, 0, 727.1492339535973, 0.8053162795963512...",43,26
2,15,288.814742,"[[63, 0, 727.1492339535973, 1.7064951032300741...",63,15
3,19,180.406473,"[[69, 0, 727.1492339535973, 1.7386094446935536...",69,19
4,1,938.734391,"[[6, 0, 727.1492339535973, 1.574772408237217, ...",6,1


In [9]:
data = df.copy()
data["item_id_action"] = [data["state_history"][i][data["action_history"][i]][1] for i in range(data.shape[0])]

for index, row in data.iterrows():
    for index,list_ in enumerate(row['state_history']):
        if(index == row['action_history'] and row['rewards_history']!=0):
            list_.extend([1])
        else:
            list_.extend([0])

In [17]:
users_items_df = []
for i in range(df.shape[0]):
    users_items_df += data['state_history'][i] 
    
users_items_df = pd.DataFrame(users_items_df,columns = ['user_id','item_id',
                                                        'price','v1','v2','v3','v4','v5',
                                                        'item_recommended'])
users_items_df

,user_id,item_id,price,v1,v2,v3,v4,v5,item_recommended
0,72,0,727.149234,1.530577,0.480074,1.541600,1.353384,-0.853064,0
1,72,1,938.734391,1.530577,0.480074,0.891951,1.329071,0.540622,0
2,72,2,849.991550,1.530577,0.480074,-1.300179,0.989033,0.442092,0
3,72,3,53.788465,1.530577,0.480074,1.821206,-0.593361,1.503941,0
4,72,4,199.061066,1.530577,0.480074,1.408624,-0.030410,0.596819,0
5,72,5,631.763552,1.530577,0.480074,0.300533,1.266337,-0.894945,0
6,72,6,340.159543,1.530577,0.480074,-0.010489,0.151493,1.021108,0
7,72,7,242.491997,1.530577,0.480074,0.495546,0.672426,-0.133566,0
8,72,8,195.664296,1.530577,0.480074,2.649243,2.343519,0.873476,0
9,72,9,904.735857,1.530577,0.480074,1.757570,0.537739,2.701148,0


In [18]:
from tffm import TFFMClassifier
import tensorflow as tf

model = TFFMClassifier()

X = users_items_df.loc[:, users_items_df.columns != 'item_recommended'].drop(['user_id','item_id'],axis=1).values
y = users_items_df.loc[:,'item_recommended'].values

In [19]:
model.fit(X, y, show_progress=True)

100%|██████████| 100/100 [00:00<00:00, 212.80epoch/s]


In [21]:
generations = 1000
params = {'user_id': USER_ID}
total_reward = 0
number_run = 3

for i in range(number_run):
    for j in range(generations):
        next_state_df = pd.DataFrame(next_state,columns = ['user_id','item_id',
                                        'price','v1','v2','v3',
                                        'v4','v5',])
        X_predict = next_state_df.drop(['user_id','item_id'],axis=1).values
        predicted_item = np.argmax(model.predict_proba(X_predict)[:,1])
        params['recommended_item'] = predicted_item

        r = requests.get(url=url_predict, params=params).json()
        reward = r['reward']
        total_reward += reward
        next_state = r['state']
        
print("Total reward: %f and Total reward per run: %f"%(total_reward, total_reward/number_run))

Total reward: 102157.002843 and Total reward per run: 34052.334281
